In [ ]:
import sys

sys.path.append('../')

from fastai.text.all import *
from fastai.vision.all import *

from mrl.imports import *
from mrl.core import *
from mrl.chem import *
from mrl.templates.all import *

from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.layers import *
from mrl.dataloaders import *
from mrl.g_models.all import *
from mrl.vocab import *
from mrl.policy_gradient import *
from mrl.train.all import *

In [3]:
# os.environ['DEFAULT_GPU'] = '1'
# os.environ['ncpus'] = '32'
torch.cuda.set_device(0)

In [4]:
# df = pd.read_csv('template_filtered.csv')
df = next(pd.read_csv('../../smiles_datasets/shard_0.csv', chunksize=20000))

In [5]:
df = df[df.smiles.map(lambda x: 8<len(x)<=100)]
df = df[df.smiles.map(lambda x: not '.' in x)]
vocab = CharacterVocab(SMILES_CHAR_VOCAB)

In [6]:
bs = 1200

train_df = df.sample(frac=0.99, random_state=42)
valid_df = df[~(df.index.isin(train_df.index))]

In [7]:
d_vocab = len(vocab.itos)
d_embedding = 256
d_hidden = 1024
n_layers = 3
input_dropout = 0.3
lstm_dropout = 0.3
bos_idx = vocab.stoi['bos']
bidir = False
tie_weights = True

model = LSTM_LM(d_vocab, 
                d_embedding,
                d_hidden, 
                n_layers,
                input_dropout,
                lstm_dropout,
                bos_idx, 
                bidir, 
                tie_weights)

In [8]:
train_ds = Text_Dataset(train_df.smiles.values, vocab)
valid_ds = Text_Dataset(valid_df.smiles.values, vocab)

train_dl = train_ds.dataloader(bs, shuffle=True, num_workers=10)
valid_dl = valid_ds.dataloader(bs, shuffle=False, num_workers=10)

In [9]:
to_device(model);

In [10]:
sys.path.append('/home/dmai/mrl/nbs/pretrain_scripts/')
from train import train_model

In [11]:
bs = 1200
lr = 1e-3
epochs = 1
loss_function = CrossEntropy()

In [12]:
train_model(model, train_dl, valid_dl, lr, epochs, bs, loss_function, 0)

Epoch,Train Loss,Valid Loss,Time
0,2.84601,2.78433,00:03


In [15]:
!python '../nbs/pretrain_scripts/lstmlm_small_zinc.py'

/home/dmai/miniconda3/envs/mrl/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)
Epoch     Train Loss  Valid  Loss  Time    
Traceback (most recent call last):                                    
  File "../nbs/pretrain_scripts/lstmlm_small_zinc.py", line 60, in <module>
    train_model(model, train_dl, valid_dl, lr, epochs, bs, loss_function, device)
  File "/home/dmai/mrl/nbs/pretrain_scripts/train.py", line 72, in train_model
    f'{valid_loss:.5f}', f'{format_time(end)}'], table=True)
  File "/home/dmai/miniconda3/envs/mrl/lib/python3.6/site-packages/fastprogress/fastprogress.py", line 280, in write
    for (t,name) in zip(line,self.names): text += t + ' ' * (2 + len(name)-len(t))
TypeError: object of type 'int' has no len()


In [16]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [ ]:
# df = next(pd.read_csv('../../smiles_datasets/shard_0.csv', chunksize=2000000))
# df = pd.read_csv('../../smiles_datasets/shard_0.csv', usecols=['smiles'])
# df = pd.read_csv('../../smiles_datasets/chembl/chembl.csv')

# df = pd.read_csv('../../smiles_datasets/single_frags.csv')

df = pd.read_csv('template_filtered.csv')

In [ ]:
df = df[df.smiles.map(lambda x: 8<len(x)<=100)]
df = df[df.smiles.map(lambda x: not '.' in x)]
vocab = CharacterVocab(SMILES_CHAR_VOCAB)

# vocab = FuncVocab(SELFIES_VOCAB, split_selfie,
#                   prefunc=smile_to_selfie, postfunc=selfie_to_smile)

In [ ]:
df.shape

(4206401, 1)

In [ ]:
bs = 1000

train_df = df.sample(frac=0.99, random_state=42)
valid_df = df[~(df.index.isin(train_df.index))]

In [ ]:
gc.collect()

0

In [ ]:
from rdkit import Chem

In [ ]:
m = to_mol('N[C@@H](C)C(=O)O')
Chem.rdmolops.RemoveStereochemistry(m)

In [ ]:
to_smile(m)

'CC(N)C(=O)O'

In [ ]:
class Text_DS_NC(Text_Dataset):
    def __init__(self, sequences, vocab, collate_function=None):
        super().__init__(sequences, vocab, collate_function)
        
    def numericalize(self, sequence):
        mol = to_mol(sequence)
        Chem.rdmolops.RemoveStereochemistry(mol)
        sequence = to_smile(mol)
        
        tokens = self.vocab.tokenize(sequence)
        ints = self.vocab.numericalize(tokens)
        ints = torch.LongTensor(ints)
        return ints

In [ ]:
train_ds = Text_DS_NC(train_df.smiles.values, vocab)
valid_ds = Text_DS_NC(valid_df.smiles.values, vocab)

train_dl = train_ds.dataloader(bs, shuffle=True, num_workers=10)
valid_dl = valid_ds.dataloader(bs, shuffle=False, num_workers=10)

dl = DataLoaders(train_dl, valid_dl)

In [ ]:
d_vocab = len(vocab.itos)
d_embedding = 256
d_hidden = 1024
n_layers = 3
input_dropout = 0.3
lstm_dropout = 0.3
bos_idx = vocab.stoi['bos']
bidir = False
tie_weights = True

model = LSTM_LM(d_vocab, 
                d_embedding,
                d_hidden, 
                n_layers,
                input_dropout,
                lstm_dropout,
                bos_idx, 
                bidir, 
                tie_weights)



In [ ]:
learn = Learner(dl, model, loss_func=CrossEntropy())

In [ ]:
learn = learn.to_fp16()

In [ ]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,time
0,0.307867,0.334042,2:42:30


In [ ]:
torch.save(learn.model.cpu().state_dict(), '../nbs/untracked_files/lstm_lm_zinc_nc.pt')

In [ ]:
train_ds = Text_Dataset(train_df.smiles.values, vocab)
valid_ds = Text_Dataset(valid_df.smiles.values, vocab)

train_dl = train_ds.dataloader(bs, shuffle=True, num_workers=10)
valid_dl = valid_ds.dataloader(bs, shuffle=False, num_workers=10)

dl = DataLoaders(train_dl, valid_dl)

In [ ]:
d_vocab = len(vocab.itos)
d_embedding = 256
d_hidden = 1024
n_layers = 3
input_dropout = 0.3
lstm_dropout = 0.3
bos_idx = vocab.stoi['bos']
bidir = False
tie_weights = True

model = LSTM_LM(d_vocab, 
                d_embedding,
                d_hidden, 
                n_layers,
                input_dropout,
                lstm_dropout,
                bos_idx, 
                bidir, 
                tie_weights)

model.load_state_dict(torch.load('../nbs/untracked_files/lstm_lm_zinc.pt'))

<All keys matched successfully>

In [ ]:
learn = Learner(dl, model, loss_func=CrossEntropy())

In [ ]:
learn = learn.to_fp16()

In [ ]:
learn.fit_one_cycle(2, 1e-4)

epoch,train_loss,valid_loss,time
0,0.339847,0.348997,04:59
1,0.330349,0.344911,04:59


In [ ]:
torch.save(learn.model.cpu().state_dict(), '../nbs/untracked_files/lstm_lm_zinc.pt')

In [ ]:
train_ds = Vec_To_Text_Dataset(train_df.smiles.values, vocab, ECFP6)
valid_ds = Vec_To_Text_Dataset(valid_df.smiles.values, vocab, ECFP6)

# train_ds = Text_Dataset(train_df.smiles.values, vocab)
# valid_ds = Text_Dataset(valid_df.smiles.values, vocab)

train_dl = train_ds.dataloader(bs, shuffle=True, num_workers=10)
valid_dl = valid_ds.dataloader(bs, shuffle=False, num_workers=10)

dl = DataLoaders(train_dl, valid_dl)

In [ ]:
d_vocab = len(vocab.itos)
d_embedding = 256
encoder_d_in = 2048
encoder_dims = [1024, 512]
encoder_drops = [0.1, 0.1]
d_hidden = 1024
d_latent = 512
n_layers = 3
input_dropout = 0.3
lstm_dropout = 0.3
condition_hidden = True
condition_output = True
bos_idx = vocab.stoi['bos']


model = MLP_VAE(
            d_vocab,
            d_embedding,
            encoder_d_in,
            encoder_dims,
            encoder_drops,
            d_hidden,
            n_layers,
            d_latent,
            input_dropout=input_dropout,
            lstm_dropout=lstm_dropout,
            condition_hidden=condition_hidden,
            condition_output=condition_output,
            prior=None,
            bos_idx=bos_idx,
            transition=None,
        )


# d_vocab = len(vocab.itos)
# d_embedding = 256
# conv_filters = [256, 512, 512, 1024]
# kernel_sizes = [7,7,7,7]
# strides = [2,2,2,2]
# conv_drops = [0.3, 0.3, 0.3,0.3]
# d_hidden = 1024
# d_latent = 512
# n_layers = 3
# input_dropout = 0.3
# lstm_dropout = 0.3
# bidir = False
# condition_hidden = True
# condition_output = True
# bos_idx = vocab.stoi['bos']

# model = Conv_VAE(
#                 d_vocab,
#                 d_embedding,
#                 conv_filters,
#                 kernel_sizes,
#                 strides,
#                 conv_drops,
#                 d_hidden,
#                 n_layers,
#                 d_latent,
#                 input_dropout=input_dropout,
#                 lstm_dropout=lstm_dropout,
#                 condition_hidden=condition_hidden,
#                 condition_output=condition_output,
#                 prior=None,
#                 bos_idx=bos_idx,
#                 transition=None,
#             )

In [ ]:
loss = VAELoss(weight=0.5)

In [ ]:
class FPVAECallback(Callback):
    "Move data to CUDA device"
    def before_batch(self): 
        self.learn.xb = self.learn.xb[0]

In [ ]:
learn = Learner(dl, model, loss_func=VAELoss(weight=0.5), cbs=[FPVAECallback()])
# learn = Learner(dl, model, loss_func=VAELoss(weight=0.5))

In [ ]:
learn.model.load_state_dict(torch.load('../nbs/untracked_files/fp_vae_zinc.pt'))

<All keys matched successfully>

In [ ]:
learn = learn.to_fp16()

In [ ]:
learn.model.forward_rollout = False

In [ ]:
learn.fit_one_cycle(2, 1e-4)

epoch,train_loss,valid_loss,time
0,0.329212,0.334790,06:23
1,0.323479,0.332716,06:22


In [ ]:
learn.model.forward_rollout = True

In [ ]:
learn.fit_one_cycle(2, 1e-4)

epoch,train_loss,valid_loss,time
0,0.608709,0.680992,13:24
1,0.611293,0.671171,13:26


In [ ]:
learn.model.p_force

1.0

In [ ]:
learn.model.force_decay = 0.998

In [ ]:
learn.fit_one_cycle(4, 1e-4)

epoch,train_loss,valid_loss,time
0,1.735332,1.745684,13:30
1,1.733853,1.732547,13:37
2,1.721871,1.735721,13:42
3,1.717928,1.733486,13:44


In [ ]:
learn.model.p_force

1.2264803595148308e-14

In [ ]:
x,y = next(iter(valid_dl))

In [ ]:
z,_ = learn.model.transition(learn.model.encoder(to_device(x[1])))

In [ ]:
z = z[0].unsqueeze(0).repeat(512,1)

In [ ]:
preds, _ = learn.model.sample_no_grad(512, 90, z=z)

In [ ]:
smiles = [vocab.reconstruct(i) for i in preds]

In [ ]:
smiles[:10]

['CCCCc(H((cccC1CC)N()CC@Hccc]=)Oc[HHHc)nccCC)C1c)H2cncn-2cc)CF]@cC',
 'CCC(c]C()CCCNC)[@HH(OCcncCCCc@(C)CCcccO)1F',
 'O=(Ncc(C(OccCc3cc1()NCCc)c])CO])N1ClCc11C',
 'CCC()1Oc[H]3ccC2(Cc=)CC1cC](CO)cCO))',
 'CC1cc(c(OCCCnc(((3[@H]C1cnnc1Cc)1C)CC=)C1C1NCCncCCc@@]CO1CCCc))c',
 'CC1C1ccNC(CC=)CC(O(OCS=)C)Cn]O2c2c)C)1CCCCC)',
 'C[@H]cC1COC[@H](cc[@@Hn1ccCccC(C(N=)NCNccc[+HH(O@cn3ccN1c)CnC(C)O1H',
 'CC([@H((1N[@H]CC1c]cCcCccccCC@H1()1[@H]C4SCc@cc(O)cc',
 'Cnc1cccC)=C]COcC(C(OCCN1C(=)Cc)cc(c)OcCC)1CO',
 'C=C1C(cccc()C((C)2cnc3N()CCcn2[HiH(]C)C)C1C[@HH((NCcc(nF(())[@@](CC)C[@H]CC()))O)C1']

In [ ]:
len([i for i in to_mols(smiles) if i is not None])

0

In [ ]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,time
0,0.352807,0.376337,4:00:59


In [ ]:
torch.save(learn.model.cpu().state_dict(), '../nbs/untracked_files/conv_vae_zinc_selfies.pt')

In [ ]:
x,y = next(iter(valid_dl))

In [ ]:
x.shape

torch.Size([500, 65])

In [ ]:
z,_ = learn.model.transition(learn.model.encoder(to_device(x)))

In [ ]:
z = z[0].unsqueeze(0).repeat(512,1)

In [ ]:
preds, _ = learn.model.sample_no_grad(512, 90, z=z)

In [ ]:
smiles = [vocab.reconstruct(i) for i in preds]

In [ ]:
smiles[:10]

['CCCOc1ccc(C(=O)N(C)c2ccc(C#N)cc2)cc1',
 'COc1cc(Nc2cccc(-n3ccnn3)c2)cc(F)c1Cl',
 'COC(=O)N1C[C@H]2CC[C@H]1[C@H](NCC(=O)Nc1cc([N+](=O)[O-])ccn1)C2',
 'Cc1cc(C(=O)N2CCC(NC(=O)[C@@H]3CN(c4ccccn4)C[C@H]3C)CC2)n[nH]1',
 'CCC[C@@H]1C[C@H]1C(=O)N[C@@H]1CN(Cc2cc(OC)ccc2Cl)C[C@@H]1O',
 'C[S@](=O)C[C@H](NC(=O)CCNS(=O)(=O)c1ccc(F)c(F)c1)c1ccccc1',
 'CCc1nnc(N2CCO[C@](C)(c3cnn(C)c3)C2)n1Cc1ccc(Cl)cc1',
 'CC[C@@H](C)Oc1cc(N2CCC[C@H]2c2ccco2)nc(C)n1',
 'O=C(O)C1CCN(c2cc(C(F)(F)F)ccn2)CC1',
 'CNS(=O)(=O)c1cc(C(=O)N2CCC(C(=O)c3cccs3)CC2)ccc1F']

In [ ]:
to_device(learn.model);

In [ ]:
df = pd.read_csv('../../smiles_datasets/chembl/chembl.csv')

In [ ]:
df = df[df.smiles.map(lambda x: 8<len(x)<=100)]
df = df[df.smiles.map(lambda x: not '.' in x)]

In [ ]:
bs = 500

train_df = df.sample(frac=0.99, random_state=42)
valid_df = df[~(df.index.isin(train_df.index))]

In [ ]:
# train_ds = Vec_To_Text_Dataset(train_df.smiles.values, vocab, ECFP6)
# valid_ds = Vec_To_Text_Dataset(valid_df.smiles.values, vocab, ECFP6)

train_ds = Text_Dataset(train_df.smiles.values, vocab)
valid_ds = Text_Dataset(valid_df.smiles.values, vocab)

train_dl = train_ds.dataloader(bs, shuffle=True, num_workers=10)
valid_dl = valid_ds.dataloader(bs, shuffle=False, num_workers=10)

dl = DataLoaders(train_dl, valid_dl)

In [ ]:
len(train_dl)

3657

In [ ]:
# learn = Learner(dl, model, loss_func=VAELoss(weight=0.5), cbs=[FPVAECallback()])
learn = Learner(dl, model, loss_func=VAELoss(weight=0.5))

In [ ]:
learn.model.load_state_dict(torch.load('../nbs/untracked_files/conv_vae_zinc_selfies.pt'))

<All keys matched successfully>

In [ ]:
learn = learn.to_fp16()

In [ ]:
learn.fit_one_cycle(10, 5e-4)

epoch,train_loss,valid_loss,time
0,0.379176,0.408467,08:09
1,0.346460,0.367585,08:02
2,0.330632,0.351018,08:03
3,0.318140,0.339556,08:04
4,0.311918,0.329689,08:03
5,0.305336,0.325988,08:03
6,0.297668,0.321791,08:03
7,0.295900,0.319025,08:03
8,0.293085,0.318507,08:04
9,0.291604,0.318038,08:05


In [ ]:
torch.save(learn.model.cpu().state_dict(), '../nbs/untracked_files/conv_vae_chembl_selfies.pt')